<a href="https://colab.research.google.com/github/joshkellett87/colab-bayesian-calc/blob/main/Bayesian_A_B_Test_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Test Calculator for Online Experiments 📈
Robust a/b/n test calculator created by Josh Kellett.

**Instructions**
1.   Go to Runtime > Run All to start calc
2.   Enter # of test variants
3.   Enter priors and set power for control + variants
4.   Choose ROPE
5.   Add sample & conversion counts for all versions

That's it! From there you'll get a detailed analysis of your test results, including decision recommendations and a full set of charts.

In [ ]:
#
# --- Boilerplate Imports & Setup for Tall Output ---
#
import sys
from io import StringIO
from IPython.display import HTML, display
import html

# Store the original standard output
_original_stdout = sys.stdout
# Create a StringIO object to capture output in memory
_captured_output = StringIO()
# Redirect standard output to our StringIO object
# This needs to happen before any libraries (like rich.Console) that might cache sys.stdout are initialized
# or before any print statements that need to be captured.
sys.stdout = _captured_output
# --- End of output capturing setup ---
#

# --- User's Original Imports ---
import numpy as np
import scipy.stats as stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from rich.console import Console # Will now write to the redirected sys.stdout
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
from rich.padding import Padding
from rich.style import Style # Included as it was in the user's script

# v=======================================================================v
# |                                                                       |
# |   USER'S FULL SCRIPT CONTENT STARTS HERE (CLASSES, FUNCTIONS, MAIN)   |
# |                                                                       |
# v=======================================================================v

# Helper function to calculate Highest Density Interval (HDI)
def _calculate_hdi(samples, credible_mass=0.95):
    """Calculate the Highest Density Interval (HDI) for a list of samples."""
    if samples is None or len(samples) == 0:
        return (np.nan, np.nan)

    samples = samples[~np.isnan(samples)]
    if len(samples) == 0:
        return (np.nan, np.nan)

    sorted_samples = np.sort(samples)
    n_samples = len(samples)

    interval_idx_inc = int(np.floor(credible_mass * n_samples))
    if interval_idx_inc == 0:
        return (np.nan, np.nan)

    n_intervals = n_samples - interval_idx_inc
    if n_intervals <= 0:
         return (sorted_samples[0], sorted_samples[-1])

    interval_width = sorted_samples[interval_idx_inc:] - sorted_samples[:n_intervals]

    if len(interval_width) == 0:
        return (sorted_samples[0], sorted_samples[-1])

    min_idx = np.argmin(interval_width)
    hdi_min = sorted_samples[min_idx]
    hdi_max = sorted_samples[min_idx + interval_idx_inc]
    return hdi_min, hdi_max

class BayesianExperiment:
    """
    A class to perform Bayesian analysis for an A/B test with binomial data,
    supporting multiple solution variants.
    """
    def __init__(self, num_solution_variants=1):
        if not isinstance(num_solution_variants, int) or num_solution_variants < 1:
            raise ValueError("Number of solution variants must be a positive integer.")
        self.num_solution_variants = num_solution_variants

        # Control parameters
        self.control_prior_alpha = 1.0
        self.control_prior_beta = 1.0
        self.control_posterior_alpha = 1.0
        self.control_posterior_beta = 1.0
        self.control_samples = 0
        self.control_conversions = 0
        self.control_observed_alpha_likelihood = 1
        self.control_observed_beta_likelihood = 1

        # Solution variant parameters (lists)
        self.solution_prior_alpha = [1.0] * num_solution_variants
        self.solution_prior_beta = [1.0] * num_solution_variants
        self.solution_posterior_alpha = [1.0] * num_solution_variants
        self.solution_posterior_beta = [1.0] * num_solution_variants
        self.solution_samples = [0] * num_solution_variants
        self.solution_conversions = [0] * num_solution_variants
        self.solution_observed_alpha_likelihood = [1] * num_solution_variants
        self.solution_observed_beta_likelihood = [1] * num_solution_variants

        self.variant_names = ["Control"] + [f"Solution {i+1}" for i in range(num_solution_variants)]


    def set_priors(self, control_alpha, control_beta, solution_alphas, solution_betas):
        if control_alpha <= 0 or control_beta <= 0:
            raise ValueError("Control prior alpha and beta parameters must be positive.")
        if not (isinstance(solution_alphas, list) and isinstance(solution_betas, list) and
                len(solution_alphas) == self.num_solution_variants and len(solution_betas) == self.num_solution_variants):
            raise ValueError(f"Solution priors must be lists of length {self.num_solution_variants}.")
        for sa, sb in zip(solution_alphas, solution_betas):
            if sa <= 0 or sb <= 0:
                raise ValueError("Solution prior alpha and beta parameters must be positive.")

        self.control_prior_alpha = control_alpha
        self.control_prior_beta = control_beta
        self.control_posterior_alpha = control_alpha
        self.control_posterior_beta = control_beta

        self.solution_prior_alpha = list(solution_alphas)
        self.solution_prior_beta = list(solution_betas)
        self.solution_posterior_alpha = list(solution_alphas)
        self.solution_posterior_beta = list(solution_betas)


    def update_results(self, control_samples, control_conversions, solution_samples_list, solution_conversions_list):
        if control_samples < control_conversions or control_samples < 0:
            raise ValueError("Control samples must be non-negative and >= control conversions.")
        if not (isinstance(solution_samples_list, list) and isinstance(solution_conversions_list, list) and
                len(solution_samples_list) == self.num_solution_variants and len(solution_conversions_list) == self.num_solution_variants):
            raise ValueError(f"Solution results must be lists of length {self.num_solution_variants}.")

        self.control_samples = control_samples
        self.control_conversions = control_conversions
        control_losses = control_samples - control_conversions
        self.control_posterior_alpha = self.control_prior_alpha + control_conversions
        self.control_posterior_beta = self.control_prior_beta + control_losses
        self.control_observed_alpha_likelihood = control_conversions + (1 if control_conversions == 0 and control_losses == 0 else 0)
        self.control_observed_beta_likelihood = control_losses + (1 if control_conversions == 0 and control_losses == 0 else 0)

        for i in range(self.num_solution_variants):
            s_samples = solution_samples_list[i]
            s_conversions = solution_conversions_list[i]
            if s_samples < s_conversions or s_samples < 0:
                raise ValueError(f"Solution {i+1} samples must be non-negative and >= conversions.")
            if s_conversions < 0:
                raise ValueError(f"Solution {i+1} conversions must be non-negative.")

            self.solution_samples[i] = s_samples
            self.solution_conversions[i] = s_conversions
            s_losses = s_samples - s_conversions
            self.solution_posterior_alpha[i] = self.solution_prior_alpha[i] + s_conversions
            self.solution_posterior_beta[i] = self.solution_prior_beta[i] + s_losses
            self.solution_observed_alpha_likelihood[i] = s_conversions + (1 if s_conversions == 0 and s_losses == 0 else 0)
            self.solution_observed_beta_likelihood[i] = s_losses + (1 if s_conversions == 0 and s_losses == 0 else 0)


    def get_posterior_samples(self, n_samples=20000):
        """Generates posterior samples for control and all solution variants."""
        all_samples = {}
        all_samples['control_rate'] = stats.beta.rvs(
            self.control_posterior_alpha, self.control_posterior_beta, size=n_samples
        )
        for i in range(self.num_solution_variants):
            variant_name = f"solution_{i+1}_rate"
            all_samples[variant_name] = stats.beta.rvs(
                self.solution_posterior_alpha[i], self.solution_posterior_beta[i], size=n_samples
            )

        for i in range(self.num_solution_variants):
            all_samples[f"abs_diff_s{i+1}_c"] = all_samples[f"solution_{i+1}_rate"] - all_samples['control_rate']

        for i in range(self.num_solution_variants):
            rel_lift_samples = np.full_like(all_samples['control_rate'], np.nan)
            valid_mask = all_samples['control_rate'] > 1e-9
            rel_lift_samples[valid_mask] = (all_samples[f"solution_{i+1}_rate"][valid_mask] - all_samples['control_rate'][valid_mask]) / all_samples['control_rate'][valid_mask]
            all_samples[f"rel_lift_s{i+1}_c"] = rel_lift_samples

        return all_samples

    def calculate_metrics(self, rope_abs_diff=(-0.005, 0.005), rope_rel_lift=(-0.05, 0.05),
                          prob_beat_threshold=0.0, credible_mass=0.95, n_samples_for_calc=20000):
        """Calculates metrics for control and all solution variants."""
        samples = self.get_posterior_samples(n_samples=n_samples_for_calc)
        metrics = {'control': {}, 'solutions': [{} for _ in range(self.num_solution_variants)]}

        control_s = samples['control_rate']
        metrics['control']['posterior_mean_rate'] = np.mean(control_s)
        metrics['control']['rate_hdi'] = _calculate_hdi(control_s, credible_mass)

        all_variant_samples = [samples['control_rate']]
        for i in range(self.num_solution_variants):
            sol_s = samples[f"solution_{i+1}_rate"]
            abs_diff_s = samples[f"abs_diff_s{i+1}_c"]
            rel_lift_s = samples[f"rel_lift_s{i+1}_c"][~np.isnan(samples[f"rel_lift_s{i+1}_c"])]

            metrics['solutions'][i]['name'] = f"Solution {i+1}"
            metrics['solutions'][i]['posterior_mean_rate'] = np.mean(sol_s)
            metrics['solutions'][i]['rate_hdi'] = _calculate_hdi(sol_s, credible_mass)

            metrics['solutions'][i]['absolute_difference_mean'] = np.mean(abs_diff_s)
            metrics['solutions'][i]['absolute_difference_hdi'] = _calculate_hdi(abs_diff_s, credible_mass)

            if len(rel_lift_s) > 0:
                metrics['solutions'][i]['relative_lift_mean'] = np.mean(rel_lift_s)
                metrics['solutions'][i]['relative_lift_hdi'] = _calculate_hdi(rel_lift_s, credible_mass)
            else:
                metrics['solutions'][i]['relative_lift_mean'] = np.nan
                metrics['solutions'][i]['relative_lift_hdi'] = (np.nan, np.nan)

            metrics['solutions'][i]['prob_beats_control'] = np.mean(sol_s > control_s)
            metrics['solutions'][i]['prob_beats_control_by_threshold'] = np.mean(sol_s > (control_s + prob_beat_threshold))

            metrics['solutions'][i]['prob_abs_diff_below_rope'] = np.mean(abs_diff_s < rope_abs_diff[0])
            metrics['solutions'][i]['prob_abs_diff_in_rope'] = np.mean((abs_diff_s >= rope_abs_diff[0]) & (abs_diff_s <= rope_abs_diff[1]))
            metrics['solutions'][i]['prob_abs_diff_above_rope'] = np.mean(abs_diff_s > rope_abs_diff[1])

            if len(rel_lift_s) > 0:
                metrics['solutions'][i]['prob_rel_lift_below_rope'] = np.mean(rel_lift_s < rope_rel_lift[0])
                metrics['solutions'][i]['prob_rel_lift_in_rope'] = np.mean((rel_lift_s >= rope_rel_lift[0]) & (rel_lift_s <= rope_rel_lift[1]))
                metrics['solutions'][i]['prob_rel_lift_above_rope'] = np.mean(rel_lift_s > rope_rel_lift[1])
            else:
                for key in ['prob_rel_lift_below_rope', 'prob_rel_lift_in_rope', 'prob_rel_lift_above_rope']:
                    metrics['solutions'][i][key] = np.nan

            metrics['solutions'][i]['expected_loss_vs_control_choosing_solution'] = np.mean(np.maximum(0, control_s - sol_s))
            metrics['solutions'][i]['expected_loss_vs_control_choosing_control'] = np.mean(np.maximum(0, sol_s - control_s))

            all_variant_samples.append(sol_s)

        stacked_samples = np.stack(all_variant_samples, axis=-1)
        best_variant_indices = np.argmax(stacked_samples, axis=1)

        metrics['prob_control_is_best'] = np.mean(best_variant_indices == 0)
        for i in range(self.num_solution_variants):
            metrics['solutions'][i]['prob_is_best'] = np.mean(best_variant_indices == (i + 1))

        return metrics

    def get_decision_summary(self, metrics, rope_abs_diff_vs_control, p_threshold=0.95, loss_ratio_threshold=5):
        """Generates a nuanced decision summary for multiple variants."""
        best_overall_prob = metrics.get('prob_control_is_best', 0.0)
        best_variant_idx = -1

        for i, sol_metrics in enumerate(metrics['solutions']):
            if sol_metrics.get('prob_is_best', 0.0) > best_overall_prob:
                best_overall_prob = sol_metrics.get('prob_is_best', 0.0)
                best_variant_idx = i

        # Case 1: Control is most likely the best
        if best_variant_idx == -1:
            evaluation = f"Control is Most Likely Best (P(Best)={metrics.get('prob_control_is_best', 0.0):.1%})"
            recommendation = "Stick with Control."
            rec_style = "blue"
            # Check if any solution offers a compelling reason to consider despite Control leading
            for i, sol_metrics in enumerate(metrics['solutions']):
                 prob_s_beats_c_by_thresh = sol_metrics.get('prob_beats_control_by_threshold', 0.0)
                 loss_ctrl_vs_sol = sol_metrics.get('expected_loss_vs_control_choosing_control', np.inf)
                 loss_sol_vs_ctrl = sol_metrics.get('expected_loss_vs_control_choosing_solution', np.inf)
                 # If a solution has high P(>Ctrl+Thresh) and significantly lower risk if chosen
                 if prob_s_beats_c_by_thresh > p_threshold and loss_ctrl_vs_sol > (loss_sol_vs_ctrl * loss_ratio_threshold / 2): # Halved threshold for more sensitivity
                      recommendation += f" However, Solution {i+1} shows strong potential (high P(>{metrics.get('prob_beat_threshold_value',0.0):.1%}) & favorable risk) and could be considered if Control's lead is marginal."
                      rec_style = "yellow" # Change style if such a solution exists
                      break
            return evaluation, recommendation, rec_style

        # Case 2: A Solution variant is most likely the best
        best_sol_metrics = metrics['solutions'][best_variant_idx]
        sol_name = best_sol_metrics['name'] # e.g. "Solution 1"
        prob_best_sol_val = best_sol_metrics.get('prob_is_best', 0.0)
        evaluation = f"{sol_name} is Most Likely Best (P(Best)={prob_best_sol_val:.1%})"

        hdi_low, hdi_high = best_sol_metrics.get('absolute_difference_hdi', (np.nan, np.nan))
        rope_low, rope_high = rope_abs_diff_vs_control

        loss_ctrl_vs_sol = best_sol_metrics.get('expected_loss_vs_control_choosing_control', np.inf)
        loss_sol_vs_ctrl = best_sol_metrics.get('expected_loss_vs_control_choosing_solution', np.inf)

        if np.isnan(hdi_low) or np.isnan(hdi_high):
            return evaluation, f"Error calculating HDI for {sol_name}. Cannot provide detailed recommendation.", "red"

        # Define helper conditions for clarity
        is_favorable_risk = loss_ctrl_vs_sol > (loss_sol_vs_ctrl * loss_ratio_threshold)

        # The p_threshold (e.g. 0.95) is for the probability of beating control *by the specified threshold*
        # The actual threshold value (e.g., 0.1% uplift) is stored in metrics['prob_beat_threshold_value']
        user_uplift_threshold_val = metrics.get('prob_beat_threshold_value', 0.0)
        prob_sol_beats_ctrl_by_user_thresh_val = best_sol_metrics.get('prob_beats_control_by_threshold', 0.0)
        is_highly_likely_to_beat_ctrl_by_thresh = prob_sol_beats_ctrl_by_user_thresh_val >= p_threshold
        threshold_text = f"Control + {user_uplift_threshold_val:.1%}" if user_uplift_threshold_val > 0 else "Control"


        # Sub-case 2.1: Clear win (HDI for difference entirely above ROPE)
        if hdi_low > rope_high:
            recommendation = f"Strongly recommend {sol_name}. Clear win vs Control (95% HDI for difference is entirely above ROPE)."
            rec_style = "green"
        # Sub-case 2.2: Clear loss (HDI for difference entirely below ROPE)
        elif hdi_high < rope_low:
            recommendation = f"Reject {sol_name}. Likely worse vs Control (95% HDI for difference is entirely below ROPE)."
            rec_style = "red"
        # Sub-case 2.3: Practically Equivalent (HDI for difference entirely within ROPE)
        elif hdi_low >= rope_low and hdi_high <= rope_high:
            if is_highly_likely_to_beat_ctrl_by_thresh and is_favorable_risk:
                 recommendation = f"Consider {sol_name}. Practically equivalent to Control (difference within ROPE), but P(>{threshold_text}) is high & risk profile is favorable."
                 rec_style = "yellow"
            elif prob_best_sol_val > 0.75 and is_favorable_risk:
                 recommendation = f"Consider {sol_name}. Practically equivalent to Control (difference within ROPE), but P(Best) is high & risk profile is favorable."
                 rec_style = "yellow"
            else:
                recommendation = f"{sol_name} is likely best but practically equivalent to Control (difference within ROPE)."
                rec_style = "blue"
        # Sub-case 2.4: Nuanced (HDI overlaps ROPE - this is the main area for tiered logic)
        else:
            if prob_best_sol_val >= 0.90:
                if is_favorable_risk or is_highly_likely_to_beat_ctrl_by_thresh:
                    recommendation = f"Strongly recommend {sol_name} (P(Best)={prob_best_sol_val:.1%}). Strong evidence and favorable risk/reward profile vs Control."
                    rec_style = "green"
                else:
                    recommendation = f"{sol_name} is very likely best (P(Best)={prob_best_sol_val:.1%}). Consider accepting vs Control."
                    rec_style = "yellow"
            elif prob_best_sol_val >= 0.75:
                if is_favorable_risk or is_highly_likely_to_beat_ctrl_by_thresh:
                    recommendation = f"{sol_name} is a strong candidate (P(Best)={prob_best_sol_val:.1%}). Good evidence and risk profile vs Control."
                    rec_style = "green"
                else:
                    recommendation = f"{sol_name} shows good promise (P(Best)={prob_best_sol_val:.1%}). Worth considering vs Control."
                    rec_style = "yellow"
            elif prob_best_sol_val >= 0.60:
                if is_favorable_risk and is_highly_likely_to_beat_ctrl_by_thresh:
                     recommendation = f"{sol_name} shows potential (P(Best)={prob_best_sol_val:.1%}). Risk/reward vs Control may be favorable."
                     rec_style = "yellow"
                elif is_favorable_risk or is_highly_likely_to_beat_ctrl_by_thresh:
                     recommendation = f"{sol_name} has some potential (P(Best)={prob_best_sol_val:.1%}). Evaluate risk/reward vs Control carefully."
                     rec_style = "yellow"
                else:
                    recommendation = f"{sol_name} is marginally ahead (P(Best)={prob_best_sol_val:.1%}). Decision vs Control requires careful consideration of other factors."
                    rec_style = "blue"
            else: # P(Best Sol) < 0.60
                recommendation = f"Evidence for {sol_name} is not conclusive (P(Best)={prob_best_sol_val:.1%}). Control remains a strong contender. Consider gathering more data if feasible."
                rec_style = "blue"

        return evaluation, recommendation, rec_style

    def _get_dynamic_axis_range(self, *distributions_params_or_samples,
                                percentile_low=0.01, percentile_high=99.99,
                                padding_factor=0.08, allow_negative=False):
        all_quantiles = np.array([])
        for item in distributions_params_or_samples:
            if item is None: continue
            if isinstance(item, tuple) and len(item) == 2:
                alpha, beta = item
                if alpha > 0 and beta > 0:
                    q_low = stats.beta.ppf(percentile_low / 100.0, alpha, beta)
                    q_high = stats.beta.ppf(percentile_high / 100.0, alpha, beta)
                    if not np.isnan(q_low) and not np.isnan(q_high):
                         all_quantiles = np.concatenate([all_quantiles, [q_low, q_high]])
            elif isinstance(item, np.ndarray) and item.size > 0:
                valid_samples = item[~np.isnan(item)]
                if valid_samples.size > 0:
                    q_low = np.percentile(valid_samples, percentile_low)
                    q_high = np.percentile(valid_samples, percentile_high)
                    all_quantiles = np.concatenate([all_quantiles, [q_low, q_high]])
        if all_quantiles.size == 0:
            return (0.0, 0.1) if not allow_negative else (-0.05, 0.05)
        min_val = np.min(all_quantiles)
        max_val = np.max(all_quantiles)
        current_range = max_val - min_val
        if current_range < 1e-6:
            padding = 0.005
        else:
            padding = current_range * padding_factor
        axis_min = min_val - padding
        if not allow_negative:
            axis_min = max(0.0, axis_min)
        axis_max = max_val + padding
        if not allow_negative:
             axis_max = min(1.0, axis_max)
        if axis_max <= axis_min:
             axis_max = axis_min + (0.001 if not allow_negative else 0.0001 * abs(axis_min) + 0.0001)
        if not allow_negative and axis_max > 1.0: axis_max = 1.0
        if not allow_negative and axis_min < 0.0: axis_min = 0.0
        return axis_min, axis_max


    def plot_distributions_plotly(self, rope_abs_diff=(-0.005, 0.005), rope_rel_lift=(-0.05, 0.05),
                                  n_samples_for_plot=10000, solution_to_compare_idx=None): # Added solution_to_compare_idx
        """
        Generate and display interactive plots for multiple variants.
        Note: Plotly fig.show() renders directly to Colab output, not captured in the tall text box.
        """
        samples_data = self.get_posterior_samples(n_samples=n_samples_for_plot)
        control_post_s = samples_data['control_rate']

        solution_line_colors = ['lightcoral', 'lightseagreen', 'mediumpurple', 'gold']
        solution_fill_colors = [
            'rgba(240,128,128,0.4)',
            'rgba(32,178,170,0.4)',
            'rgba(147,112,219,0.4)',
            'rgba(255,215,0,0.4)'
        ]

        metrics_temp = self.calculate_metrics(rope_abs_diff, rope_rel_lift)
        best_sol_idx_for_plot = -1
        max_p_best = metrics_temp.get('prob_control_is_best', 0.0)
        best_sol_name_for_title = "Solution 1"
        if self.num_solution_variants > 0:
            # Ensure there's at least one solution to avoid index error if num_solution_variants is 0
            if metrics_temp['solutions']:
                best_sol_name_for_title = metrics_temp['solutions'][0]['name']

        for i, sol_metrics in enumerate(metrics_temp['solutions']):
            if sol_metrics.get('prob_is_best', 0.0) > max_p_best:
                max_p_best = sol_metrics.get('prob_is_best', 0.0)
                best_sol_idx_for_plot = i
                best_sol_name_for_title = sol_metrics['name']

        # Override with user selection if provided
        if solution_to_compare_idx is not None and 0 <= solution_to_compare_idx < self.num_solution_variants:
            best_sol_idx_for_plot = solution_to_compare_idx
            best_sol_name_for_title = f"Solution {best_sol_idx_for_plot + 1}"
        elif solution_to_compare_idx is not None: # Invalid index provided
            # console.print(f"[yellow]Warning: Invalid solution_to_compare_idx ({solution_to_compare_idx}). Using best performer or Solution 1.[/yellow]")
            # Fallback to best_sol_idx_for_plot determined above, or Solution 1 if control was best
            if best_sol_idx_for_plot == -1 and self.num_solution_variants > 0:
                 best_sol_idx_for_plot = 0 # Default to Solution 1 for diff plots if control is best
                 best_sol_name_for_title = f"Solution 1"


        if best_sol_idx_for_plot == -1 and self.num_solution_variants > 0 :
             best_sol_idx_for_plot = 0 # Default to Solution 1 for diff plots if control is best
             best_sol_name_for_title = f"Solution 1"
        elif self.num_solution_variants == 0: # No solutions to plot differences for
            best_sol_name_for_title = "N/A"


        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                "<b>Prior Distributions</b>",
                "<b>Observed Data Likelihoods</b>",
                "<b>Posterior Distributions</b>",
                f"<b>Which Variant is Most Likely the Winner?</b>",
                f"<b>Absolute Difference: {best_sol_name_for_title} vs. Control</b>",
                f"<b>Probability of {best_sol_name_for_title} Beating Control by > X% (Relative Lift)</b>"
            ),
            specs=[[{}, {}],
                   [{}, {}],
                   [{}, {}]],
            vertical_spacing=0.15,
            horizontal_spacing=0.1
        )

        all_prior_params = [(self.control_prior_alpha, self.control_prior_beta)] + \
                           [(self.solution_prior_alpha[i], self.solution_prior_beta[i]) for i in range(self.num_solution_variants)]
        prior_min_x, prior_max_x = self._get_dynamic_axis_range(*all_prior_params, allow_negative=False)
        x_prior_plot = np.linspace(prior_min_x, prior_max_x, 200)

        all_like_params = [(self.control_observed_alpha_likelihood, self.control_observed_beta_likelihood) if self.control_samples > 0 else None] + \
                          [(self.solution_observed_alpha_likelihood[i], self.solution_observed_beta_likelihood[i]) if self.solution_samples[i] > 0 else None for i in range(self.num_solution_variants)]
        like_min_x, like_max_x = self._get_dynamic_axis_range(*[p for p in all_like_params if p is not None], allow_negative=False)
        x_like_plot = np.linspace(like_min_x, like_max_x, 200)

        all_post_samples = [control_post_s] + [samples_data[f"solution_{i+1}_rate"] for i in range(self.num_solution_variants) if f"solution_{i+1}_rate" in samples_data]
        post_min_x, post_max_x = self._get_dynamic_axis_range(*[s for s in all_post_samples if s is not None and len(s) > 0], allow_negative=False)
        x_post_plot = np.linspace(post_min_x, post_max_x, 200)

        # Plot 1: Prior Distributions
        fig.add_trace(go.Scatter(x=x_prior_plot, y=stats.beta.pdf(x_prior_plot, self.control_prior_alpha, self.control_prior_beta),
                                 mode='lines', name='Ctrl Prior', legendgroup="Priors", line=dict(dash='dash', color='skyblue', width=2),
                                 hovertemplate="<b>Ctrl Prior</b><br>Rate: %{x:.3%}<br>Density: %{y:.2f}<extra></extra>"), row=1, col=1)
        for i in range(self.num_solution_variants):
            fig.add_trace(go.Scatter(x=x_prior_plot, y=stats.beta.pdf(x_prior_plot, self.solution_prior_alpha[i], self.solution_prior_beta[i]),
                                     mode='lines', name=f'Sol {i+1} Prior', legendgroup="Priors",
                                     line=dict(dash='dash', color=solution_line_colors[i % len(solution_line_colors)], width=2),
                                     hovertemplate=f"<b>Sol {i+1} Prior</b><br>Rate: %{{x:.3%}}<br>Density: %{{y:.2f}}<extra></extra>"), row=1, col=1)
        fig.update_xaxes(range=[prior_min_x, prior_max_x], row=1, col=1)

        # Plot 2: Observed Data Likelihoods
        if self.control_samples > 0:
            fig.add_trace(go.Scatter(x=x_like_plot, y=stats.beta.pdf(x_like_plot, self.control_observed_alpha_likelihood, self.control_observed_beta_likelihood),
                                     mode='lines', name='Ctrl Likelihood', legendgroup="Likelihoods", line=dict(dash='dot', color='lightgreen', width=2),
                                     hovertemplate="<b>Ctrl Likelihood</b><br>Rate: %{x:.3%}<br>Density: %{y:.2f}<extra></extra>"), row=1, col=2)
        for i in range(self.num_solution_variants):
            if self.solution_samples[i] > 0:
                fig.add_trace(go.Scatter(x=x_like_plot, y=stats.beta.pdf(x_like_plot, self.solution_observed_alpha_likelihood[i], self.solution_observed_beta_likelihood[i]),
                                         mode='lines', name=f'Sol {i+1} Likelihood', legendgroup="Likelihoods",
                                         line=dict(dash='dot', color=solution_line_colors[i % len(solution_line_colors)], width=1.5),
                                         opacity=0.8,
                                         hovertemplate=f"<b>Sol {i+1} Likelihood</b><br>Rate: %{{x:.3%}}<br>Density: %{{y:.2f}}<extra></extra>"), row=1, col=2)
        if self.control_samples == 0 and all(s == 0 for s in self.solution_samples):
             fig.add_annotation(text="No observed data entered", showarrow=False, row=1, col=2)
        fig.update_xaxes(range=[like_min_x, like_max_x], row=1, col=2)

        # Plot 3: Posterior Distributions
        # max_density_post = 0 # Not used, can be removed
        if control_post_s is not None and len(control_post_s) > 1:
            kde_control = stats.gaussian_kde(control_post_s)
            y_kde_control = kde_control(x_post_plot)
            # max_density_post = max(max_density_post, np.max(y_kde_control) if len(y_kde_control)>0 else 0)
            fig.add_trace(go.Scatter(x=x_post_plot, y=y_kde_control, mode='lines', name='Ctrl Posterior', legendgroup="Posteriors", fill='tozeroy',
                                     fillcolor='rgba(70,130,180,0.4)', line=dict(color='steelblue', width=2),
                                     hovertemplate="<b>Ctrl Posterior</b><br>Rate: %{x:.3%}<br>Density: %{y:.2f}<extra></extra>"), row=2, col=1)
        for i in range(self.num_solution_variants):
            sol_s_key = f"solution_{i+1}_rate"
            if sol_s_key in samples_data:
                sol_s = samples_data[sol_s_key]
                if sol_s is not None and len(sol_s) > 1:
                    kde_solution = stats.gaussian_kde(sol_s)
                    y_kde_solution = kde_solution(x_post_plot)
                    # max_density_post = max(max_density_post, np.max(y_kde_solution) if len(y_kde_solution)>0 else 0)
                    fig.add_trace(go.Scatter(x=x_post_plot, y=y_kde_solution, mode='lines', name=f'Sol {i+1} Posterior', legendgroup="Posteriors", fill='tozeroy',
                                            fillcolor=solution_fill_colors[i % len(solution_fill_colors)],
                                            line=dict(color=solution_line_colors[i % len(solution_line_colors)], width=2),
                                            hovertemplate=f"<b>Sol {i+1} Posterior</b><br>Rate: %{{x:.3%}}<br>Density: %{{y:.2f}}<extra></extra>"), row=2, col=1)
        fig.update_xaxes(range=[post_min_x, post_max_x], row=2, col=1)

        # Plot 4: Probability of Being Best
        prob_best_names = [self.variant_names[0]] + [sol_metrics['name'] for sol_metrics in metrics_temp['solutions']]
        prob_best_values = [metrics_temp.get('prob_control_is_best', 0)] + [sol_metrics.get('prob_is_best', 0) for sol_metrics in metrics_temp['solutions']]
        bar_colors = ['skyblue'] + [solution_line_colors[i % len(solution_line_colors)] for i in range(self.num_solution_variants)]
        fig.add_trace(go.Bar(x=prob_best_names, y=prob_best_values, name='P(Best)', legendgroup="P(Best)",
                             marker_color=bar_colors, text=[f"{p:.1%}" for p in prob_best_values], textposition='auto',
                             hovertemplate="<b>%{x}</b><br>P(Best): %{y:.2%}<extra></extra>"), row=2, col=2)
        fig.update_yaxes(tickformat=".0%", range=[0,1.05], row=2, col=2)

        # Plot 5: Difference (Selected/Best Solution vs Control)
        selected_sol_abs_diff_s = np.array([])
        if best_sol_idx_for_plot != -1 and f"abs_diff_s{best_sol_idx_for_plot+1}_c" in samples_data:
            selected_sol_abs_diff_s = samples_data[f"abs_diff_s{best_sol_idx_for_plot+1}_c"]

        if len(selected_sol_abs_diff_s) > 1:
            diff_min_x, diff_max_x = self._get_dynamic_axis_range(selected_sol_abs_diff_s, allow_negative=True)
            x_diff_plot = np.linspace(diff_min_x, diff_max_x, 200)
            kde_abs_diff = stats.gaussian_kde(selected_sol_abs_diff_s)
            y_kde_abs_diff = kde_abs_diff(x_diff_plot)
            fig.add_trace(go.Scatter(x=x_diff_plot, y=y_kde_abs_diff, mode='lines', name=f'Abs. Diff ({best_sol_name_for_title})', legendgroup="Difference Analysis", fill='tozeroy',
                                     fillcolor='rgba(128,0,128,0.4)', line=dict(color='purple', width=2),
                                     hovertemplate="<b>Abs. Difference</b><br>Value: %{x:.3%}<br>Density: %{y:.2f}<extra></extra>"), row=3, col=1)
            abs_diff_mean = np.mean(selected_sol_abs_diff_s)
            abs_diff_hdi = _calculate_hdi(selected_sol_abs_diff_s)
            fig.add_vline(x=abs_diff_mean, line_width=1.5, line_dash="dash", line_color="indigo", row=3, col=1)
            fig.add_vline(x=abs_diff_hdi[0], line_width=1.5, line_dash="dot", line_color="indigo", row=3, col=1)
            fig.add_vline(x=abs_diff_hdi[1], line_width=1.5, line_dash="dot", line_color="indigo", row=3, col=1)
            if rope_abs_diff: # Check if rope_abs_diff is defined
                fig.add_shape(type="rect", x0=rope_abs_diff[0], x1=rope_abs_diff[1], y0=0, y1=np.max(y_kde_abs_diff)*1.1 if len(y_kde_abs_diff) > 0 else 1,
                            fillcolor="rgba(169,169,169,0.3)", opacity=0.3, layer="below", line_width=0, name="ROPE Abs.Diff.", row=3, col=1)
            fig.update_xaxes(range=[diff_min_x, diff_max_x], row=3, col=1)
        elif self.num_solution_variants > 0 : # Only add annotation if solutions exist but no data for this plot
            fig.add_annotation(text=f"No data for Absolute Difference plot ({best_sol_name_for_title})", showarrow=False, row=3, col=1)
        else: # No solutions at all
            fig.add_annotation(text="No solution variants to compare.", showarrow=False, row=3, col=1)


        # Plot 6: Cumulative P(Selected/Best Solution Relative Uplift > X)
        selected_sol_rel_lift_s = np.array([])
        if best_sol_idx_for_plot != -1 and f"rel_lift_s{best_sol_idx_for_plot+1}_c" in samples_data:
            selected_sol_rel_lift_s = samples_data[f"rel_lift_s{best_sol_idx_for_plot+1}_c"][~np.isnan(samples_data[f"rel_lift_s{best_sol_idx_for_plot+1}_c"])]

        if len(selected_sol_rel_lift_s) > 0:
            cum_rel_min_x, cum_rel_max_x = self._get_dynamic_axis_range(selected_sol_rel_lift_s, allow_negative=True)
            sorted_rel_lift = np.sort(selected_sol_rel_lift_s)
            y_cumulative_rel = 1. - (np.arange(len(sorted_rel_lift)) / float(len(sorted_rel_lift)))
            fig.add_trace(go.Scatter(x=sorted_rel_lift, y=y_cumulative_rel, mode='lines', name=f'P(Rel. Uplift ({best_sol_name_for_title}) > X)', legendgroup="Cumulative Uplift", line=dict(color='darkcyan', width=2),
                                     hovertemplate="<b>P(Rel. Uplift > X)</b><br>Rel. Uplift (X): %{x:.2%}<br>Probability: %{y:.2%}<extra></extra>"), row=3, col=2)
            fig.add_hline(y=0.95, line_width=1, line_dash="dash", line_color="gray", row=3, col=2)
            fig.add_hline(y=0.50, line_width=1, line_dash="dot", line_color="gray", row=3, col=2)
            fig.add_vline(x=0, line_width=1, line_dash="solid", line_color="black", row=3, col=2)
            fig.update_xaxes(range=[cum_rel_min_x, cum_rel_max_x], row=3, col=2)
            fig.update_yaxes(range=[0,1.05], row=3, col=2)
        elif self.num_solution_variants > 0:
             fig.add_annotation(text=f"Not enough data for Cumulative Rel. Uplift ({best_sol_name_for_title})", showarrow=False, row=3, col=2)
        else:
             fig.add_annotation(text="No solution variants to compare.", showarrow=False, row=3, col=2)


        fig.update_layout(
            height=1200,
            title_text="<b>Bayesian A/B Test Visualizations</b>", title_x=0.5, title_font_size=20,
            legend_traceorder='grouped', legend_tracegroupgap=15, hovermode='x unified', template='plotly_white'
        )
        for r_idx,c_idx in [(1,1),(1,2),(2,1),(2,2),(3,1),(3,2)]: fig.update_xaxes(tickformat=".2%", row=r_idx, col=c_idx) # Use r_idx, c_idx to avoid conflict
        for r_idx,c_idx in [(1,1),(1,2),(2,1),(3,1)]: fig.update_yaxes(title_text="Density", row=r_idx, col=c_idx) # Use r_idx, c_idx
        fig.update_yaxes(title_text="Probability P(Best)", tickformat=".0%", row=2, col=2)
        fig.update_yaxes(title_text="Probability P(Rel. Uplift > X)", tickformat=".0%", row=3, col=2)

        fig.show() # This will render directly in Colab output, not in the HTML box

    def plot_forest_hdi(self, metrics):
        """
        Generates and displays a forest plot of the 95% HDIs for conversion rates.
        Note: Plotly fig.show() renders directly to Colab output, not captured in the tall text box.
        """
        variant_names = ["Control"] + [sol['name'] for sol in metrics['solutions']]
        mean_rates = [metrics['control']['posterior_mean_rate']] + [sol['posterior_mean_rate'] for sol in metrics['solutions']]
        hdi_lows = [metrics['control']['rate_hdi'][0]] + [sol['rate_hdi'][0] for sol in metrics['solutions']]
        hdi_highs = [metrics['control']['rate_hdi'][1]] + [sol['rate_hdi'][1] for sol in metrics['solutions']]

        control_color = 'rgba(70,130,180,0.8)'
        solution_base_colors = ['rgba(240,128,128,0.8)', 'rgba(32,178,170,0.8)', 'rgba(147,112,219,0.8)', 'rgba(255,215,0,0.8)']
        variant_colors = [control_color] + [solution_base_colors[i % len(solution_base_colors)] for i in range(self.num_solution_variants)]

        fig = go.Figure()

        for i in range(len(variant_names)):
            # Skip if HDI is nan, which can happen with no samples
            if np.isnan(hdi_lows[i]) or np.isnan(hdi_highs[i]):
                continue
            fig.add_trace(go.Scatter(
                x=[hdi_lows[i], hdi_highs[i]],
                y=[variant_names[i], variant_names[i]],
                mode='lines',
                line=dict(color=variant_colors[i], width=2),
                name=f"{variant_names[i]} 95% HDI",
                legendgroup=variant_names[i],
                showlegend=False,
                hovertemplate=f"<b>{variant_names[i]}</b><br>95% HDI: [{hdi_lows[i]:.2%}, {hdi_highs[i]:.2%}]<extra></extra>"
            ))
            if not np.isnan(mean_rates[i]): # Also check mean_rate for NaN
                fig.add_trace(go.Scatter(
                    x=[mean_rates[i]],
                    y=[variant_names[i]],
                    mode='markers',
                    marker=dict(color=variant_colors[i], size=10, symbol='diamond'),
                    name=variant_names[i],
                    legendgroup=variant_names[i],
                    hovertemplate=f"<b>{variant_names[i]}</b><br>Mean Rate: {mean_rates[i]:.2%}<br>95% HDI: [{hdi_lows[i]:.2%}, {hdi_highs[i]:.2%}]<extra></extra>"
                ))

        if not np.isnan(metrics['control']['posterior_mean_rate']):
            fig.add_vline(
                x=metrics['control']['posterior_mean_rate'],
                line_width=1, line_dash="dash", line_color="grey",
                annotation_text="Control Mean", annotation_position="bottom right"
            )

        fig.update_layout(
            title="<b>Credible Conversion Rates (95% HDI)</b>",
            title_x=0.5,
            xaxis_title="Conversion Rate",
            yaxis_title="Variant",
            yaxis=dict(autorange="reversed"),
            template='plotly_white',
            height=150 + (len(variant_names) * 50),
            hovermode='closest',
            legend_title_text='Variants'
        )
        fig.update_xaxes(tickformat=".2%")
        fig.show() # This will render directly in Colab output, not in the HTML box


# --- Display Helper Functions --- (These use `console.print` which will be captured)
def display_test_outcomes_table(console, metrics):
    """Displays the Test Outcomes table for multiple variants."""
    table = Table(title="Test Outcomes Summary", title_style="bold magenta", border_style="blue")
    table.add_column("Group", style="cyan")
    table.add_column("Win Rate (Mean)", style="dim")
    table.add_column("Rel. Lift vs Ctrl (Mean)", style="dim")
    table.add_column("95% HDI (Rate)", style="dim")

    c_metrics = metrics['control']
    c_rate_hdi_low, c_rate_hdi_high = c_metrics.get('rate_hdi', (np.nan, np.nan))
    table.add_row("Control",
                  f"{c_metrics.get('posterior_mean_rate', np.nan):.2%}",
                  "N/A",
                  f"[{c_rate_hdi_low:.2%}, {c_rate_hdi_high:.2%}]")

    for sol_metrics in metrics['solutions']:
        rl_mean = sol_metrics.get('relative_lift_mean', np.nan)
        rate_hdi_low, rate_hdi_high = sol_metrics.get('rate_hdi', (np.nan, np.nan))
        table.add_row(
            sol_metrics.get('name', 'N/A'),
            f"{sol_metrics.get('posterior_mean_rate', np.nan):.2%}",
            f"{rl_mean:+.2%}" if not np.isnan(rl_mean) else "N/A",
            f"[{rate_hdi_low:.2%}, {rate_hdi_high:.2%}]"
        )
    console.print(Padding(table, (1, 0)))


def display_confidence_intervals_summary(console, metrics):
    """Displays a dedicated summary of key confidence intervals for multiple variants."""
    panel_content = Text()
    c_metrics = metrics['control']
    c_rate_hdi_low, c_rate_hdi_high = c_metrics.get('rate_hdi', (np.nan, np.nan))
    panel_content.append("Control Conversion Rate:\n", style="bold sky_blue3")
    panel_content.append(f"  Mean: {c_metrics.get('posterior_mean_rate', np.nan):.2%}, 95% HDI: [{c_rate_hdi_low:.2%}, {c_rate_hdi_high:.2%}]\n\n")

    for sol_metrics in metrics['solutions']:
        sol_name = sol_metrics.get('name', 'N/A')
        sol_rate_hdi_low, sol_rate_hdi_high = sol_metrics.get('rate_hdi', (np.nan, np.nan))
        panel_content.append(f"{sol_name} Conversion Rate:\n", style="bold light_coral")
        panel_content.append(f"  Mean: {sol_metrics.get('posterior_mean_rate', np.nan):.2%}, 95% HDI: [{sol_rate_hdi_low:.2%}, {sol_rate_hdi_high:.2%}]\n\n")

        abs_diff_hdi_low, abs_diff_hdi_high = sol_metrics.get('absolute_difference_hdi', (np.nan, np.nan))
        panel_content.append(f"Abs. Diff ({sol_name} - Control):\n", style="bold dark_violet")
        panel_content.append(f"  Mean: {sol_metrics.get('absolute_difference_mean', np.nan):.2%}, 95% HDI: [{abs_diff_hdi_low:.2%}, {abs_diff_hdi_high:.2%}]\n")

        rl_mean_val = sol_metrics.get('relative_lift_mean', np.nan)
        rl_hdi_low_val, rl_hdi_high_val = sol_metrics.get('relative_lift_hdi', (np.nan, np.nan))
        if not np.isnan(rl_mean_val):
            panel_content.append(f"Rel. Lift (({sol_name}-Ctrl)/Ctrl):\n", style="bold green4")
            panel_content.append(f"  Mean: {rl_mean_val:.2%}, 95% HDI: [{rl_hdi_low_val:.2%}, {rl_hdi_high_val:.2%}]\n\n")
        else:
            panel_content.append(f"Rel. Lift (({sol_name}-Ctrl)/Ctrl): N/A (Control rate might be zero or too low)\n\n")

    console.print(Panel(panel_content, title="[bold]Confidence Intervals (95% HDI)[/bold]", border_style="steel_blue", expand=False))


def display_detailed_metrics(console, metrics, rope_abs_diff, rope_rel_lift):
    panel_content = Text()
    prob_beat_thresh_val_display = metrics.get('prob_beat_threshold_value', 0.0)

    panel_content.append("Probability of Being Best Overall:\n", style="bold underline")
    panel_content.append(f"  Control: {metrics.get('prob_control_is_best', 0.0):.2%}\n")
    for sol_metrics in metrics['solutions']:
        panel_content.append(f"  {sol_metrics.get('name','N/A')}: {sol_metrics.get('prob_is_best', 0.0):.2%}\n")
    panel_content.append("\n")

    for i, sol_metrics in enumerate(metrics['solutions']):
        sol_name = sol_metrics.get('name','N/A')
        panel_content.append(f"--- Analysis for {sol_name} vs Control ---\n", style="bold yellow")
        panel_content.append("Probabilities:\n", style="bold underline")
        panel_content.append(f"  P({sol_name} > Control): {sol_metrics.get('prob_beats_control',np.nan):.2%}\n")
        panel_content.append(f"  P({sol_name} > Control + {prob_beat_thresh_val_display:.1%}): {sol_metrics.get('prob_beats_control_by_threshold',np.nan):.2%}\n\n")

        if rope_abs_diff: # Check if ROPE for absolute difference is defined
            panel_content.append(f"ROPE Analysis (Absolute Difference: {rope_abs_diff[0]:.2%} to {rope_abs_diff[1]:.2%}):\n", style="bold underline")
            panel_content.append(f"  P(Diff < ROPE Low): {sol_metrics.get('prob_abs_diff_below_rope',np.nan):.2%}\n")
            panel_content.append(f"  P(Diff In ROPE):   {sol_metrics.get('prob_abs_diff_in_rope',np.nan):.2%}\n")
            panel_content.append(f"  P(Diff > ROPE High):{sol_metrics.get('prob_abs_diff_above_rope',np.nan):.2%}\n\n")
        else:
            panel_content.append("ROPE Analysis (Absolute Difference): Not applicable (e.g. control rate is zero).\n\n")


        if not np.isnan(sol_metrics.get('prob_rel_lift_in_rope', np.nan)) and rope_rel_lift:
            panel_content.append(f"ROPE Analysis (Relative Lift: {rope_rel_lift[0]:.1%} to {rope_rel_lift[1]:.1%}):\n", style="bold underline")
            panel_content.append(f"  P(Lift < ROPE Low): {sol_metrics.get('prob_rel_lift_below_rope', np.nan):.2%}\n")
            panel_content.append(f"  P(Lift In ROPE):   {sol_metrics.get('prob_rel_lift_in_rope', np.nan):.2%}\n")
            panel_content.append(f"  P(Lift > ROPE High):{sol_metrics.get('prob_rel_lift_above_rope', np.nan):.2%}\n\n")
        else:
            panel_content.append("ROPE Analysis (Relative Lift): Not applicable (e.g. control rate is zero or ROPE not defined).\n\n")


        panel_content.append(f"Expected Loss ({sol_name} vs Control):\n", style="bold underline")
        panel_content.append(f"  Choosing {sol_name} (if Control is better): {sol_metrics.get('expected_loss_vs_control_choosing_solution',np.nan):.4%}\n")
        panel_content.append(f"  Choosing Control (if {sol_name} is better): {sol_metrics.get('expected_loss_vs_control_choosing_control',np.nan):.4%}\n\n")

    console.print(Panel(panel_content, title="[bold]Further Analysis Details[/bold]", border_style="green", expand=False))

def display_explanations(console):
    text = Text()
    text.append("Key Concepts:\n\n", style="bold underline")
    text.append("ROPE (Region of Practical Equivalence):\n", style="bold cyan")
    text.append("  The range of differences you consider too small to matter. If the credible interval for the difference falls mostly within ROPE, the variants are practically equivalent.\n\n")
    text.append("HDI (Highest Density Interval):\n", style="bold cyan")
    text.append("  The range containing a specific percentage (e.g., 95%) of the most credible values for a parameter (e.g., conversion rate or difference). We can say there's a 95% probability the true value lies within the 95% HDI.\n\n")
    text.append("Interpreting 'Further Analysis Details':\n", style="bold underline")
    text.append("  - Probability of Being Best: For each variant, the chance it has the highest true conversion rate among all tested variants (including Control).\n")
    text.append("  - Probabilities (vs Control): Shows the likelihood of a solution variant being better than Control, or better by a certain threshold.\n")
    text.append("  - ROPE Analysis (vs Control): Shows the probability that the true difference/lift between a solution and Control falls below, within, or above your defined ROPE.\n")
    text.append("  - Expected Loss (vs Control): Estimates the average 'cost' of making the wrong decision between a specific solution and Control.\n\n")
    text.append("Interpreting Charts:\n", style="bold underline")
    text.append("  - Prior plots show initial beliefs for Control and all Solutions, overlaid.\n")
    text.append("  - Likelihood plots show what current test data suggests for each variant, overlaid.\n")
    text.append("  - Posterior plots combine priors and likelihood for updated beliefs, overlaid. HDIs are marked as small shaded regions at the base.\n")
    text.append("  - P(Best) Bar Chart: Visualizes the probability of each variant being the overall best.\n")
    text.append("  - Difference plots show distributions of (Solution - Control) for the best solution or a selected one.\n")
    text.append("  - Cumulative P(Uplift > X) plot shows the probability that the true relative uplift (for the best solution vs Control) is greater than X.\n")
    text.append("  - Forest Plot: Compares the 95% HDIs of conversion rates for all variants side-by-side.\n")
    text.append("  - Hover over chart elements for specific values.\n")
    console.print(Panel(text, title="[bold]Understanding the Results[/bold]", border_style="magenta", expand=False))


if __name__ == "__main__":
    # --- Colab Form Inputs ---
    # @title Bayesian A/B Test Analyzer Inputs
    # @markdown ### General Setup
    Number_of_Solution_Variants = 1 #@param {type:"integer", min:1, max:5, step:1}

    # @markdown ---
    # @markdown ### Prior Input Method
    # @markdown Choose how to define your prior beliefs. "Assumed Rate & Strength" is generally more intuitive.
    Prior_Input_Method = "Assumed Rate & Strength (Recommended)" #@param ["Assumed Rate & Strength (Recommended)", "Direct Alpha & Beta (Advanced)"]

    # @markdown ---
    # @markdown ### Priors: Control Group
    # @markdown Based on your chosen input method:
    Control_Assumed_Conversion_Rate = 0.10 #@param {type:"number"}
    Control_Prior_Strength_Pseudo_Observations = 100 #@param {type:"integer"}
    # @markdown ---
    # @markdown *Advanced: Direct Alpha/Beta for Control (only used if "Direct Alpha & Beta" method is selected above)*
    Control_Prior_Alpha_Advanced = 1.0 #@param {type:"number"}
    Control_Prior_Beta_Advanced = 1.0 #@param {type:"number"}

    # @markdown ---
    # @markdown ### Priors: Solution Variant(s)
    Auto_Derive_Solution_Priors_From_Control_Rate = True #@param {type:"boolean"}
    # @markdown *If auto-deriving: Uses Control's Assumed Rate. Enter Strength (CSV for multiple, e.g., "20" or "20,15").*
    Solution_Priors_Strength_CSV_Auto = "100" #@param {type:"string"}
    # @markdown *If NOT auto-deriving (and using Rate & Strength method): Enter Assumed Rates (CSV, e.g., "0.12,0.15") and Strengths (CSV, e.g., "20,25") for each solution.*
    Solution_Assumed_Rates_CSV_Manual = "0.12" #@param {type:"string"}
    Solution_Priors_Strength_CSV_Manual = "20" #@param {type:"string"}
    # @markdown ---
    # @markdown *Advanced: Direct Alpha/Beta for Solutions (CSV, e.g., "1,1"). Only used if "Direct Alpha & Beta" method is selected.*
    Solution_Prior_Alphas_Advanced_CSV = "1.0" #@param {type:"string"}
    Solution_Prior_Betas_Advanced_CSV = "1.0" #@param {type:"string"}

    # @markdown ---
    # @markdown ### Test Results
    # @markdown **Control Group:**
    Control_Group_Samples = 6000 #@param {type:"integer"}
    Control_Group_Conversions = 600 #@param {type:"integer"}
    # @markdown **Solution Group(s):**
    # @markdown *Enter comma-separated values if multiple solutions (e.g., `1000,1010` for samples).*
    Solution_Samples_CSV = "6000" #@param {type:"string"}
    Solution_Conversions_CSV = "610" #@param {type:"string"}

    # @markdown ---
    # @markdown ### ROPE (Region of Practical Equivalence)
    ROPE_Definition_Method = "Relative Lift (%)" #@param ["Relative Lift (%)", "Absolute Difference (Decimal)"]
    # @markdown *Enter a positive value for the symmetrical boundary. E.g., for +/-2%, enter 2.*
    ROPE_Relative_Lift_Symmetrical_Boundary_Percent = 1.0 #@param {type:"number"}
    # @markdown *Enter a positive decimal for the symmetrical boundary. E.g., for +/-0.5%, enter 0.005.*
    ROPE_Absolute_Difference_Symmetrical_Boundary_Decimal = 0.005 #@param {type:"number"}

    # @markdown ---
    # @markdown ### Decision Making Parameters
    # @markdown *Probability threshold for P(Solution > Control + Uplift Threshold) to be considered 'high enough' for stronger recommendations.*
    P_Beats_Control_Threshold_for_Decision = 0.95 #@param {type:"number", min:0.5, max:0.999, step:0.01}
    # @markdown *Minimum Uplift Threshold (decimal, e.g., 0.001 for 0.1%) for calculating P(Solution > Control + Threshold). This is the 'X' in P(Sol > Ctrl + X).*
    Min_Uplift_Threshold_Decimal_for_Prob_Calc = 0.000 #@param {type:"number"}
    # @markdown *Loss Ratio Threshold: How many times greater must Expected Loss of Choosing Control (if Solution is better) be, compared to Expected Loss of Choosing Solution (if Control is better), to consider the risk profile 'favorable' for the Solution? (e.g., 5 means 5x)*
    Loss_Ratio_Threshold_for_Favorable_Risk = 5.0 #@param {type:"number", min:1.0}


    # @markdown ---
    # @markdown ### Plot Settings
    # @markdown *Select which Solution variant's difference plots to display. Default is the one with highest P(Best).*
    # Dynamically create dropdown options for solution comparison plot
    solution_plot_options_list = ["Best Performer (Default)"] + [f"Solution {i+1}" for i in range(Number_of_Solution_Variants)]
    Variant_to_Display_in_Difference_Plots = "Best Performer (Default)" #@param ["Best Performer (Default)"] {allow-input: true}

    # Initialize console here, after stdout redirection and before any prints.
    # The Console instance will pick up the redirected sys.stdout.
    console = Console()

    try:
        # --- Process Form Inputs ---
        num_solution_variants = int(Number_of_Solution_Variants)

        solution_prior_alphas = []
        solution_prior_betas = []

        if Prior_Input_Method == "Assumed Rate & Strength (Recommended)":
            control_prior_rate = float(Control_Assumed_Conversion_Rate)
            control_prior_strength = int(Control_Prior_Strength_Pseudo_Observations)
            if not (0 <= control_prior_rate <= 1): raise ValueError("Control Assumed Conversion Rate must be between 0 and 1.")
            if control_prior_strength < 2: raise ValueError("Control Prior Strength must be at least 2.")
            control_prior_alpha = control_prior_rate * control_prior_strength
            control_prior_beta = (1 - control_prior_rate) * control_prior_strength
            # Ensure alpha and beta are at least 1 for a proper Beta distribution
            if control_prior_alpha < 1.0: control_prior_alpha = 1.0; control_prior_beta = float(max(1.0, control_prior_strength - 1.0))
            if control_prior_beta < 1.0: control_prior_beta = 1.0; control_prior_alpha = float(max(1.0, control_prior_strength - 1.0))

            if Auto_Derive_Solution_Priors_From_Control_Rate:
                try:
                    strengths_csv = Solution_Priors_Strength_CSV_Auto.split(',')
                    if len(strengths_csv) == 1 and num_solution_variants > 1: strengths_csv = [strengths_csv[0]] * num_solution_variants # Apply single value to all
                    if len(strengths_csv) != num_solution_variants: raise ValueError(f"Solution Priors Strength CSV count ({len(strengths_csv)}) must match Number of Solution Variants ({num_solution_variants}).")
                    solution_prior_strengths = [int(s.strip()) for s in strengths_csv]
                except Exception as e: raise ValueError(f"Invalid Solution_Priors_Strength_CSV_Auto format: {e}")

                for strength in solution_prior_strengths:
                    if strength < 2: raise ValueError("Solution Prior Strength must be at least 2.")
                    s_alpha = control_prior_rate * strength # Use control_prior_rate for auto-derivation
                    s_beta = (1-control_prior_rate) * strength
                    if s_alpha < 1.0: s_alpha = 1.0; s_beta = float(max(1.0, strength - 1.0))
                    if s_beta < 1.0: s_beta = 1.0; s_alpha = float(max(1.0, strength - 1.0))
                    solution_prior_alphas.append(s_alpha)
                    solution_prior_betas.append(s_beta)
            else: # Manual rate & strength for solutions
                try:
                    rates_csv = Solution_Assumed_Rates_CSV_Manual.split(',')
                    strengths_csv = Solution_Priors_Strength_CSV_Manual.split(',')
                    if len(rates_csv) == 1 and num_solution_variants > 1: rates_csv = [rates_csv[0]] * num_solution_variants
                    if len(strengths_csv) == 1 and num_solution_variants > 1: strengths_csv = [strengths_csv[0]] * num_solution_variants
                    if len(rates_csv) != num_solution_variants or len(strengths_csv) != num_solution_variants:
                        raise ValueError(f"Manual Solution Assumed Rates/Strengths CSV counts must match Number of Solution Variants ({num_solution_variants}).")
                    solution_assumed_rates = [float(r.strip()) for r in rates_csv]
                    solution_prior_strengths = [int(s.strip()) for s in strengths_csv]
                except Exception as e: raise ValueError(f"Invalid format for manual Solution Assumed Rates/Strengths CSV: {e}")

                for i in range(num_solution_variants):
                    rate = solution_assumed_rates[i]
                    strength = solution_prior_strengths[i]
                    if not (0 <= rate <= 1): raise ValueError(f"Solution {i+1} Assumed Rate must be 0-1.")
                    if strength < 2: raise ValueError(f"Solution {i+1} Prior Strength must be >= 2.")
                    s_alpha = rate * strength
                    s_beta = (1-rate) * strength
                    if s_alpha < 1.0: s_alpha = 1.0; s_beta = float(max(1.0, strength - 1.0))
                    if s_beta < 1.0: s_beta = 1.0; s_alpha = float(max(1.0, strength - 1.0))
                    solution_prior_alphas.append(s_alpha)
                    solution_prior_betas.append(s_beta)
        else: # Direct Alpha/Beta input
            control_prior_alpha = float(Control_Prior_Alpha_Advanced)
            control_prior_beta = float(Control_Prior_Beta_Advanced)
            if control_prior_alpha <=0 or control_prior_beta <=0: raise ValueError("Advanced Control Priors (Alpha, Beta) must be > 0.")
            try:
                alphas_csv = Solution_Prior_Alphas_Advanced_CSV.split(',')
                betas_csv = Solution_Prior_Betas_Advanced_CSV.split(',')
                if len(alphas_csv) == 1 and num_solution_variants > 1: alphas_csv = [alphas_csv[0]] * num_solution_variants
                if len(betas_csv) == 1 and num_solution_variants > 1: betas_csv = [betas_csv[0]] * num_solution_variants

                solution_prior_alphas = [float(x.strip()) for x in alphas_csv]
                solution_prior_betas = [float(x.strip()) for x in betas_csv]

                if len(solution_prior_alphas) != num_solution_variants or len(solution_prior_betas) != num_solution_variants:
                    raise ValueError(f"Advanced Solution Alpha/Beta CSV counts must match Number of Solution Variants ({num_solution_variants}).")
                for sa, sb in zip(solution_prior_alphas, solution_prior_betas):
                    if sa <=0 or sb <=0: raise ValueError("Advanced Solution Priors (Alpha, Beta) must be > 0.")
            except Exception as e: raise ValueError(f"Invalid format for Advanced Solution Alpha/Beta CSV: {e}")

        # Test Results
        control_samples = int(Control_Group_Samples)
        control_conversions = int(Control_Group_Conversions)
        if control_samples < 0 or control_conversions < 0 or control_conversions > control_samples:
            raise ValueError("Invalid Control Group samples/conversions.")
        try:
            samples_csv = Solution_Samples_CSV.split(',')
            conversions_csv = Solution_Conversions_CSV.split(',')
            if len(samples_csv) == 1 and num_solution_variants > 1: samples_csv = [samples_csv[0]] * num_solution_variants
            if len(conversions_csv) == 1 and num_solution_variants > 1: conversions_csv = [conversions_csv[0]] * num_solution_variants

            solution_samples_list = [int(s.strip()) for s in samples_csv]
            solution_conversions_list = [int(c.strip()) for c in conversions_csv]

            if len(solution_samples_list) != num_solution_variants or len(solution_conversions_list) != num_solution_variants:
                raise ValueError(f"Number of solution samples/conversions entries ({len(solution_samples_list)}/{len(solution_conversions_list)}) must match Number of Solution Variants ({num_solution_variants}).")
            for i in range(num_solution_variants):
                if solution_samples_list[i] < 0 or solution_conversions_list[i] < 0 or solution_conversions_list[i] > solution_samples_list[i]:
                     raise ValueError(f"Invalid samples/conversions for Solution {i+1}.")
        except ValueError as e: # Catch specific conversion errors from int() or logic checks
            raise ValueError(f"Invalid format or values for solution samples/conversions. Use comma-separated integers and ensure conversions <= samples. Error: {e}")
        except Exception as e: # Catch other unexpected errors during parsing
            raise ValueError(f"Unexpected error parsing solution samples/conversions: {e}")


        # ROPE
        rope_abs_diff = None
        rope_rel_lift = None
        # Calculate observed control rate, ensure it's float division
        control_observed_rate = (float(control_conversions) / control_samples) if control_samples > 0 else 0.0

        if ROPE_Definition_Method == "Relative Lift (%)":
            rel_bound = float(ROPE_Relative_Lift_Symmetrical_Boundary_Percent) / 100.0
            if rel_bound < 0: raise ValueError("ROPE Relative Lift Symmetrical Boundary Percent must be non-negative.")
            rope_rel_lift = (-rel_bound, rel_bound)
            if control_observed_rate > 1e-9: # Avoid division by zero or near-zero
                abs_delta = rel_bound * control_observed_rate
                rope_abs_diff = (-abs_delta, abs_delta)
            else:
                console.print("[yellow]Warning: Control rate is effectively 0. Cannot derive Absolute ROPE from Relative Lift. Absolute ROPE comparisons will be skipped.[/yellow]")
                # rope_abs_diff remains None
        else: # Absolute Difference (Decimal)
            abs_bound = float(ROPE_Absolute_Difference_Symmetrical_Boundary_Decimal)
            if abs_bound < 0: raise ValueError("ROPE Absolute Difference Symmetrical Boundary Decimal must be non-negative.")
            rope_abs_diff = (-abs_bound, abs_bound)
            if control_observed_rate > 1e-9:
                rel_delta = abs_bound / control_observed_rate
                rope_rel_lift = (-rel_delta, rel_delta)
            else:
                console.print("[yellow]Warning: Control rate is effectively 0. Cannot derive Relative ROPE from Absolute Difference. Relative ROPE comparisons will be skipped.[/yellow]")
                # rope_rel_lift remains None

        # Decision making parameters from form
        p_beats_control_decision_thresh = float(P_Beats_Control_Threshold_for_Decision)
        min_uplift_for_prob_calc = float(Min_Uplift_Threshold_Decimal_for_Prob_Calc)
        loss_ratio_decision_thresh = float(Loss_Ratio_Threshold_for_Favorable_Risk)
        if loss_ratio_decision_thresh < 1.0: raise ValueError("Loss Ratio Threshold must be >= 1.0.")


        solution_to_compare_idx_for_plot = None
        if Variant_to_Display_in_Difference_Plots != "Best Performer (Default)":
            try:
                selected_solution_name = Variant_to_Display_in_Difference_Plots.strip()
                # solution_plot_options_list is defined above the form, ensure it's up-to-date if variants change
                current_plot_options = ["Best Performer (Default)"] + [f"Solution {i+1}" for i in range(num_solution_variants)]
                if selected_solution_name in current_plot_options:
                    raw_index = current_plot_options.index(selected_solution_name)
                    if raw_index > 0:
                        solution_to_compare_idx_for_plot = raw_index - 1
                    # else: Best Performer, so solution_to_compare_idx_for_plot remains None (handled in plot func)
                else:
                    console.print(f"[yellow]Warning: Value '{selected_solution_name}' for 'Variant to Display in Difference Plots' is not a recognized option. Defaulting to best performer.[/yellow]")
            except ValueError:
                 console.print(f"[yellow]Warning: Value '{Variant_to_Display_in_Difference_Plots}' for 'Variant to Display in Difference Plots' is not in the generated options. Defaulting to best performer.[/yellow]")
            except Exception as e:
                console.print(f"[yellow]Could not parse '{Variant_to_Display_in_Difference_Plots}' for comparison plot. Defaulting to best performer. Error: {e}[/yellow]")

        experiment = BayesianExperiment(num_solution_variants=num_solution_variants)
        experiment.set_priors(
            control_prior_alpha, control_prior_beta,
            solution_prior_alphas, solution_prior_betas
        )
        experiment.update_results(
            control_samples, control_conversions,
            solution_samples_list, solution_conversions_list
        )

        console.print("\n[bold]Calculating metrics...[/bold]\n")
        metrics = experiment.calculate_metrics(
            rope_abs_diff=rope_abs_diff if rope_abs_diff is not None else (-0.001, 0.001), # Provide a default if None
            rope_rel_lift=rope_rel_lift if rope_rel_lift is not None else (-0.01, 0.01), # Provide a default if None
            prob_beat_threshold=min_uplift_for_prob_calc # This is the X in P(Sol > Ctrl + X)
        )
        # Store the actual uplift threshold value used for display in recommendations
        metrics['prob_beat_threshold_value'] = min_uplift_for_prob_calc

        evaluation, recommendation, rec_style = experiment.get_decision_summary(
            metrics,
            rope_abs_diff_vs_control=rope_abs_diff, # Pass the potentially None ROPE
            p_threshold=p_beats_control_decision_thresh, # This is the 0.95 like threshold
            loss_ratio_threshold=loss_ratio_decision_thresh
        )
        summary_panel_text = Text()
        summary_panel_text.append("Evaluation: ", style="bold")
        summary_panel_text.append(f"{evaluation}\n", style=f"bold {rec_style}")
        summary_panel_text.append("Recommendation: ", style="bold")
        summary_panel_text.append(f"{recommendation}", style=f"bold {rec_style}")
        console.print(Panel(summary_panel_text, title="[bold blue]Decision Summary[/bold blue]", expand=False, border_style=rec_style))

        display_test_outcomes_table(console, metrics)
        display_confidence_intervals_summary(console, metrics)
        display_detailed_metrics(console, metrics, rope_abs_diff, rope_rel_lift)

        console.print(Panel(Text("Visualizations (Plotly charts will render below this text box)", justify="center"), title="[bold]Charts[/bold]", border_style="yellow", expand=False))
        experiment.plot_distributions_plotly(
            rope_abs_diff=rope_abs_diff if rope_abs_diff is not None else (-0.001, 0.001),
            rope_rel_lift=rope_rel_lift if rope_rel_lift is not None else (-0.01, 0.01),
            solution_to_compare_idx=solution_to_compare_idx_for_plot
        )

        console.print(Panel(Text("Credible Conversion Rates (95% HDI) - Forest Plot (Plotly chart will render below)", justify="center"), title="[bold]Forest Plot[/bold]", border_style="yellow", expand=False))
        experiment.plot_forest_hdi(metrics)

        display_explanations(console)
        console.print("\n[bold green]Analysis Complete.[/bold green]")

    except ValueError as ve:
        console.print(f"[bold red]Input Error:[/bold red] {ve}")
    except Exception as e:
        console.print(f"[bold red]An unexpected error occurred:[/bold red] {e}")
        import traceback
        console.print(traceback.format_exc())

# ^=======================================================================^
# |                                                                       |
# |    USER'S FULL SCRIPT CONTENT ENDS HERE                               |
# |                                                                       |
# ^=======================================================================^
#

#
# --- Boilerplate: Display captured output in a tall HTML box ---
#
sys.stdout = _original_stdout # Restore the original standard output

output_content = _captured_output.getvalue() # Get all the content that was "printed"

_captured_output.close() # Close the StringIO object

# Escape the captured output to safely embed it in HTML
# This prevents issues if your output contains characters like <, >, &
escaped_output = html.escape(output_content)

# --- Configuration for the output display ---
output_max_height = "1200px" # Maximum height for the scrollable text area
# Removed min-height property entirely to allow the box to shrink to content size
output_bg_color = "#f9f9f9"
output_border_color = "#d0d0d0"
output_padding = "20px"
output_font_size = "13px"
output_line_height = "1.6"

# Create the HTML structure
# - The outer `div` uses max-height to grow with content up to a limit.
# - By omitting min-height, it should shrink to content size (plus padding).
# - `overflow-y: auto` enables a vertical scrollbar if content exceeds max-height.
# - The inner `<pre>` tag ensures that your output's formatting (line breaks, spaces from rich) is preserved.
html_to_display = f"""
<div style="max-height: {output_max_height};
            overflow-y: auto;
            border: 1px solid {output_border_color};
            padding: {output_padding};
            background-color: {output_bg_color};
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.05);
            ">
    <pre style="margin: 0;
                white-space: pre-wrap; /* Handles rich library's formatting */
                word-wrap: break-word;
                font-family: 'Menlo', 'Consolas', 'Monaco', 'Liberation Mono', 'Lucida Console', monospace;
                font-size: {output_font_size};
                line-height: {output_line_height};
                color: #333;
                "><code>{escaped_output}</code></pre>
</div>
"""

# Display the HTML in the Colab output area
display(HTML(html_to_display))
# --- End of output display boilerplate ---
